# AES Plaintext Recovery (Feedforward Neural Network)
In this experiment, the network tries to guess the plaintext from the ciphertext, helped with ascii per-byte correction. 

## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from dataset.datasets import AESDatasetCiphertextPlaintext
from pipeline import *

2023-05-11 17:52:47.504106: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Importing the dataset

In [43]:
data = AESDatasetCiphertextPlaintext(128, 'small')

train_labels, train_samples, test_labels, test_samples = data.get_data()

In [94]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

===== Training Labels Shape: (189568, 128)
===== Label Shape: (128,)
===== Training Samples Shape: (189568, 128)
===== Sample Shape: (128,)
===== Testing Labels Shape: (81243, 128)
===== Testing Samples Shape: (81243, 128)


## Creating the model

In [95]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense, BatchNormalization, LayerNormalization
from keras.optimizers import Adam

### Model hyperparameters
In this code block, we specify most parameters and hyperparameters that will be used in the training of the neural network.

In [190]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = len(train_labels[0])

# units per hidden layer
units = dim*16

loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
# 0.1 to 0.001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=500,
    decay_rate=0.01)
optimizer = Adam(learning_rate=0.001)
metrics = ['accuracy', 'binary_accuracy']
epochs = 500
batch_size = 5000

### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

In [191]:
# Type of model
neural_network = Sequential()

# Input layer
neural_network.add(Input(shape=input_shape))

# Hidden layers
#neural_network.add(BatchNormalization())
neural_network.add(Dense(units=units, activation='relu'))



# Output layer
neural_network.add(Dense(units=dim, activation='sigmoid'))

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_mse, metrics=metrics)

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_73 (Dense)            (None, 2048)              264192    
                                                                 
 dense_74 (Dense)            (None, 128)               262272    
                                                                 
Total params: 526,464
Trainable params: 526,464
Non-trainable params: 0
_________________________________________________________________


### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [192]:
history = train_model(neural_network, train_samples, train_labels, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/500
35/35 [==============================] - 3s 82ms/step - loss: 0.1784 - accuracy: 0.0159 - binary_accuracy: 0.7258 - val_loss: 0.1714 - val_accuracy: 0.0051 - val_binary_accuracy: 0.7358
Epoch 2/500
35/35 [==============================] - 3s 78ms/step - loss: 0.1719 - accuracy: 0.0020 - binary_accuracy: 0.7347 - val_loss: 0.1709 - val_accuracy: 1.0550e-04 - val_binary_accuracy: 0.7363
Epoch 3/500
35/35 [==============================] - 3s 76ms/step - loss: 0.1715 - accuracy: 1.2895e-04 - binary_accuracy: 0.7350 - val_loss: 0.1707 - val_accuracy: 0.0034 - val_binary_accuracy: 0.7364
Epoch 4/500
35/35 [==============================] - 3s 77ms/step - loss: 0.1714 - accuracy: 0.0163 - binary_accuracy: 0.7351 - val_loss: 0.1707 - val_accuracy: 0.0130 - val_binary_accuracy: 0.7367
Epoch 5/500
35/35 [==============================] - 3s 79ms/step - loss: 0.1713 - accuracy: 0.0112 - binary_accuracy: 0.7351 - val_loss: 0.1707 - val_accuracy: 0.0191 - val_binary_accuracy: 0.7366
Ep

In [193]:
print("Loss: {}".format(history.history['loss']))
print("Validation Loss: {}".format(history.history['val_loss']))
print("Validation Accuracy: {}".format(history.history['val_accuracy']))

Loss: [0.17835503816604614, 0.17191094160079956, 0.1715185046195984, 0.17138563096523285, 0.17129413783550262, 0.1712343841791153, 0.17116862535476685, 0.17110799252986908, 0.17104437947273254, 0.1709853708744049, 0.17092542350292206, 0.17085878551006317, 0.1707763373851776, 0.17069244384765625, 0.1706109493970871, 0.17053823173046112, 0.1704282909631729, 0.17033255100250244, 0.17022258043289185, 0.17011727392673492, 0.17000745236873627, 0.16989445686340332, 0.16977965831756592, 0.169644296169281, 0.16951817274093628, 0.16938146948814392, 0.16925641894340515, 0.16911514103412628, 0.16897818446159363, 0.16886913776397705, 0.16869136691093445, 0.16859444975852966, 0.16844305396080017, 0.16830983757972717, 0.16817842423915863, 0.16804607212543488, 0.1679316759109497, 0.1677711009979248, 0.16765011847019196, 0.16751159727573395, 0.1673957109451294, 0.16725893318653107, 0.16717064380645752, 0.16703304648399353, 0.16687709093093872, 0.16676048934459686, 0.16663573682308197, 0.166494548320770

### Testing
Here, we evaluate the neural network with the test data.

This block stays the same.

### Prediction
Here is where we use the network as an attack. We could skip the testing phase and use this as our own testing phase. Here we could add the text correction "layer" and calculate the actual score we want, maybe using binary accuracy probability as a metric.

In [197]:
results = neural_network.evaluate(test_samples, test_labels, batch_size=batch_size)
print("Test loss: {}".format(results[0]))
print("Test accuracy: {}".format(results[1]))

17/17 [==============================] - 0s 26ms/step - loss: 0.1868 - accuracy: 0.0127 - binary_accuracy: 0.7068
Test loss: 0.1868186742067337
Test accuracy: 0.012702632695436478


In [208]:
pred_size = 1000
predictions = [predict_sample(neural_network, test_samples[i]) for i in range(pred_size)]

1/1 [==============================] - 0s 20ms/step


In [209]:
metrics = [correct_and_metrics((predictions[i], test_labels[i])) for i in range(pred_size)]

In [210]:
correct_bytes = 0
correct_predictions = 0
for m in metrics:
    correct_bytes += m[0]
    correct_predictions += m[1]
num_bytes = len(test_labels[0]) // 8
                             
print("Correct bytes: {}".format(correct_bytes))
print("Byte accuracy: {}".format(correct_bytes/(num_bytes*pred_size)))
print("Correct predictions: {}".format(correct_predictions))
print("Prediction accuracy: {}".format(correct_predictions/pred_size))

Correct bytes: 752
Byte accuracy: 0.047
Correct predictions: 0
Prediction accuracy: 0.0


In [211]:
for i in range(pred_size):
    print(prediction_to_string(test_labels[i]) + " || " + prediction_to_string(predictions[i]))

:33 And when thi || he u`dd mm `jar`
s cometh to pass || d``jd g(v aed`en
, (lo, it will c || fd hed``dd`e`a`e
ome,) then shall || `d```aee adiv ao

they know that  || d```e`daad```ab`
a prophet hath b || de(u(uhe``a`u`d`
een among them.
 || b```d`e```ae`ahe

34:1 And the wo || d`aohen``md dhe`
rd of the LORD c || ` `le `md``ae`eb
ame unto me, say || ad fd `md d(ap``
ing, 34:2 Son of || `ab` ao  `o$ bme
 man,
prophesy a ||  mo00ae``mb`e``d
gainst the sheph || `ddd`a`e`aamn```
erds of Israel,  ||  dl`d`ge``he` ej
prophesy, and sa || `eld`omd``eb``he
y unto them,
Thu || !on$ die``m`amh`
s saith the Lord || `o`a`daed`iod  `
 GOD unto the sh || amd`ad`ehdd`od `
epherds; Woe be  || `ejd `eg  aed ``
to the shepherds || hm$`oc``hd`ed``m
 of
Israel that  || `d` aed``aedddd`
do feed themselv || (nl `l0 aed`hmd`
es! should not t || aee`de `edd`gm``
he shepherds fee || ``e``ag```oa`h``
d the
flocks?  3 || elpbd`d``a`aeddd
4:3 Ye eat the f ||  mne `e u(emf` e
at, and ye cloth || d``ed `he dao` h
e

In [207]:
save_model(neural_network, 'aes_feedforward')